In [1]:
import re
import enum
from math import *
from dataclasses import dataclass

@dataclass
class Group:
    id: int
    team: str
    ini: int
    units: int
    hp: int
    damage: int
    damage_type: str
    weak: list
    immune: list
    target: int
    targetedby: int
    name: str=""
    
#parse line to group
def pg(l,s,id_counter):
    #print(l)
    r={}
    
    units=re.search("[0-9]+ units",l).group(0)
    units=re.search("[0-9]+",units).group(0)
    units=int(units)
    #print("units",units)
    r["units"]=units
    
    hp=re.search("[0-9]+ hit points",l).group(0)
    hp=re.search("[0-9]+",hp).group(0)
    hp=int(hp)
    #print("hp",hp)
    r["hp"]=hp
    
    dmg=re.search("[0-9]+ [a-z]+ damage",l).group(0)
    dmg_typ=dmg.split(' ')[1]
    dmg=re.search("[0-9]+",dmg).group(0)
    dmg=int(dmg)
    #print("dmg",dmg,dmg_typ)
    r["dmg"]=dmg
    
    ini=re.search("initiative [0-9]*",l).group(0)
    ini=re.search("[0-9]+",ini).group(0)
    ini=int(ini)
    #print("ini",ini)
    r["ini"]=ini
    
    
    quirks={}
    quirks["weak"]=[]
    quirks["immune"]=[]
    
            
    for q in re.findall("\([a-z; ,]+\)",l):
        q = q[1:-1]
        #print(q)
        for p in q.split("; "):
            #print("p",p)
            category=p.split(" to ")
            #print("category", category)
            p=category[1].split(", ")
            for pp in p:
                quirks[category[0]].append(pp)
    
    #print(quirks)
            
    #print("quirks",quirks)
    for k in quirks.keys():
        r[k]=quirks[k]
        
    g = Group(id_counter, s,ini, units, hp, dmg, dmg_typ, quirks["weak"], quirks["immune"], -1, -1)
    id_counter += 1
    #print(g)
    return g

#load file into dictionary of groups
def d(fn):
    #global id_counter
    id_counter = 0
    groups = []
    cnt = {}
    s=None
    for l in open(fn).readlines():
        if len(l)>3:
            if ':' in l:
                s=l.split(':')[0].split(" ")[0]
                if s not in cnt.keys():
                    cnt[s]=1
            else:
                g = pg(l,s,id_counter)
                g.name = "Group %d"%(cnt[s])
                cnt[s]+=1
                groups.append(g)
                id_counter+=1
    G={}
    for g in groups:
        G[g.id] = g
    return G

#print group
def p(g):
    print(g.team, g.name, g.units, g.damage)

#effective power
def ep(g):
    r = g.units*g.damage
    if r<0:
        r=0
    return r 

#compute damage between attacker and target
def damage(attacker,target):
    assert(attacker.team != target.team)
    d = ep(attacker)
    if attacker.damage_type in target.immune:
        #print("IMMUNE")
        d*=0
    elif attacker.damage_type in target.weak:
        #print("WEAK")
        d*=2
    return d

#Fight till the end    
def fight(G):
    dbg = 0
    round = 0
    stale=0
    while 1:
        if dbg:
            print("-----------------------")
            print("round %d"%(round))
            for i in range(len(G.keys())):
                if G[i].units:
                    p(G[i])

        #clear attack flags
        for k in G.keys():
            G[k].target = -1
            G[k].targetedby = -1

        if dbg:
             print("### selections ###")

        #let groups select targets after effective damage and initiative if tied
        for a in [y.id for y in sorted(G.values(), key=lambda x: ep(x)*100+x.ini, reverse=True)]:
            #print("--->",ep(G[a]))
            if G[a].units > 0:
                #find potential targets
                r = "%s %s selects: " % (G[a].team, G[a].name)

                targets = [z for z in G.values() if z.team != G[a].team  and z.units > 0 and z.targetedby==-1]
                targets = sorted(targets, key=lambda x: damage(G[a], x)*100000000000 + ep(x)*100 + x.ini, reverse=True)
                targets = [x.id for x in targets]
                #print(targets)
                for t in targets:
                    dmgg=damage(G[a], G[t])
                    if dmgg>0:
                        #print(t)
                        r += "%s %s"%(G[t].team, G[t].name)
                        r += " damage %2d"%(dmgg)
                        G[a].target     = t
                        G[t].targetedby = a
                    break
                if dbg:
                    print(r)
        if dbg:
             print("### attacks ###")
        kills=0
        #perform attacks 
        for a in [x.id for x in sorted(G.values(), key=lambda y: y.ini, reverse=True)]:
            #print()
            t = G[a].target
            if t >= 0:
                #print("%d attacking %d"%(a, G[a].target))
                #only alive can attack 
                if G[a].units > 0 and G[t].units > 0:
                    s = "%s %s attacking %s %s"% (G[a].team, G[a].name, G[t].team, G[t].name)
                    hps = damage(G[a], G[t])
                    s += " with damage %d "%(hps)
                    unitloss = 0
                    while hps >= G[t].hp and G[t].units > 0:
                        hps-=G[t].hp
                        G[t].units -= 1
                        unitloss+=1
                    s += "causing %d units to be lost" % (unitloss)
                    kills+=unitloss
                    if G[t].units == 0:
                        s += " fatally"
                    if dbg:
                        print(s)
        if kills==0:
            return None,None,round
        
        if dbg:
            for i in range(len(G.keys())):
                if G[i].units:
                    p(G[i])
        immune_groups = sum([1 for x in G.values() if "Immune" in x.team and x.units > 0])
        infection_groups = sum([1 for x in G.values() if "Infect" in x.team and x.units > 0])
        if immune_groups == 0 or infection_groups==0:
            #print("Immune groups:    %d"%(immune_groups))
            #print("Infection groups: %d"%(infection_groups))
            w = sum([x.units for x in G.values()])
            return immune_groups > infection_groups, w, round
        round+=1
    return None, None, round


print("test1: ", fight(d("24a.txt"))[1], 5216)
print("part1: ", fight(d("24.txt"))[1], 18346)
        
    

test1:  5216 5216
part1:  18346 18346


In [2]:
def part2(fn):
    
    boost = 1
    while 1:
        
        print("Trying boost", boost, end="\r")
        G = d(fn)
        for i in G.keys():
            if "Imm" in G[i].team: 
                G[i].damage += boost
        r,u,rnds=fight(G)
        #if r is None:
            #print("\ntie", boost, rnds)
        if r:
            #print("\n\n", u)
            return u
        boost+=1
        
print("\ntest2: ", part2("24a.txt"), 51)
print("\npart2: ", part2("24.txt"), 8698)
print("\nfin")

Trying boost 1570
test2:  51 51
Trying boost 29
part2:  8698 8698

fin


In [3]:
#9175 too high